In [211]:
import cv2 as cv
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [212]:
# img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_23.jpg"
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/imagini_auxiliare/01.jpg"
img1 = cv.imread(img_path)
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_01.jpg"
img2 = cv.imread(img_path)
templates_path = '../templates/'
ALL_DIGITS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]

DIGIT_DICT = {digit: 7 if digit > 0 and digit < 11 else 1 for digit in ALL_DIGITS}

BASE_MATRIX = np.ones((14,14),dtype=np.int16)
BASE_MATRIX = BASE_MATRIX *-1
BASE_MATRIX[6,6] = 1
BASE_MATRIX[6,7] = 2
BASE_MATRIX[7,6] = 3
BASE_MATRIX[7,7] = 4

In [213]:
def show_img(img,title):
    if img.shape[0] > 1000:
        img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    cv.imshow(title,img)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Use hsv colors to diferentiate the board from the rest of the background and get a mask

In [214]:
def crop_image_hsv(img):
    # img = cv.imread(img_path)
    # img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    # low_yellow = (14, 0, 0)
    # high_yellow = (255, 255, 255)
    # low_yellow = (0, 0, 0)
    # high_yellow = (14, 255, 255)
    low_yellow = (17, 0, 0)
    high_yellow = (116, 255, 255)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)

    # kernel = np.ones((1,1),np.uint8)
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
    # cv.imshow("graay",mask_yellow_hsv)
    # image_bi_blur = cv.bilateralFilter(img, 15, 75, 75) 
    # gray = cv.cvtColor(image_bi_blur,cv.COLOR_BGR2GRAY)
    # print(mask_yellow_hsv.shape)
    # print(gray.shape)
    # mask_yellow_hsv = cv.bitwise_not(mask_yellow_hsv)
    # pre = cv.bitwise_and(gray,gray,mask=mask_yellow_hsv)
    # crop_img=cv.resize(crop_img,(0,0),fx=0.20,fy=0.20)
    # crop_img = extrage_careu(crop_img)
    # crop_img = cv.cvtColor(crop_img,cv.COLOR_BAYER_BG2GRAY)
    return mask_yellow_hsv

### Using the mask from the above function return only the board

In [215]:
def extrage_careu(original_img, cropped_mask_img):
    # image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(image,3)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    # image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    # #show_image('image_sharpened',image_sharpened)
    # _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    # show_img(cropped_mask_img,'cropped_mask_img')
    kernel = np.ones((3, 4), np.uint8)
    cropped_mask_img = cv.erode(cropped_mask_img, kernel)
    
    # show_img(thresh,'image_thresholded')
    

    edges =  cv.Canny(cropped_mask_img,20,400)
    # cv.imshow("canny",edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2800
    height = 2800
    
    # image_copy = cv.cvtColor(original_img.copy(),cv.COLOR_GRAY2BGR)
    image_copy = original_img.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),1)
    # cv.imshow("detected corners",image_copy)
    # show_img(image_copy,"A")

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(original_img, M, (width, height))
    # result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    
    return result

### Get the difference between the 2 imgs

In [216]:
def get_boards_difference(resized_img1,resized_img2):
    crop_img1 = crop_image_hsv(resized_img1)
    careu_img1 = extrage_careu(original_img=resized_img1,cropped_mask_img=crop_img1)

    crop_img2 = crop_image_hsv(resized_img2)
    careu_img2 = extrage_careu(original_img=resized_img2,cropped_mask_img=crop_img2)
    
    # kernel = np.ones((3,3),np.uint8)
    result = cv.absdiff(careu_img2,careu_img1)

    # result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

    return crop_img1,careu_img1,crop_img2,careu_img2,result
        

### Generate the lines that delimitate the squares in the cropped picture

In [217]:
def generate_lines():
    lines_horizontal=[]
    for i in range(370,2452,148):
        l=[]
        l.append((350,i))
        l.append((2440,i))
        lines_horizontal.append(l)
    
    lines_vertical=[]
    for i in range(357,2600,148):
        l=[]
        l.append((i,370))
        l.append((i,2440))
        lines_vertical.append(l)

    return lines_horizontal,lines_vertical

### find the coordinates of the new places piece

In [218]:
def find_new_piece_position(thresh,lines_horizontal,lines_vertical):
    # matrix = np.empty((9,9), dtype='str')
    max_i = 0
    max_j = 0
    max_medie = 0
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            Medie_patch=np.mean(patch)
            if Medie_patch > max_medie:
                max_i = i
                max_j = j
                max_medie = Medie_patch
    return max_i,max_j

### Extract the patch with the new piece and preprocess it for the template matching

In [219]:
def extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img):
    patch = careu_img[lines_horizontal[i][0][1]:lines_horizontal[i+1][0][1],lines_vertical[j][0][0]:lines_vertical[j+1][0][0],:]

    patch = patch[5:-5,5:-5,:]

    patch_blur = cv.bilateralFilter(patch, 15, 75, 75)

    patch_gray = cv.cvtColor(patch_blur,cv.COLOR_BGR2GRAY)

    # patch_blur = cv.bilateralFilter(patch_gray, 15, 75, 75)

    # patch_threshAdap = cv.adaptiveThreshold(patch_gray, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 5, 2)
    T, patch_threshOtsu = cv.threshold(patch_gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
    # _, patch_threshClassis = cv.threshold(patch_gray, 99, 255, cv.THRESH_BINARY_INV)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3))

    patch_open = cv.morphologyEx(patch_threshOtsu, cv.MORPH_OPEN, kernel)


    # _, threshClassis1 = cv.threshold(gray, 95, 255, cv.THRESH_BINARY)
    return patch_open

### Get the templates for each individual number

In [220]:
def write_digits_template(img,lines_horizontal,lines_vertical):
    index = 0
    for i in range(0,len(lines_horizontal)-2,2):
        for j in range(0,len(lines_vertical)-2,2):
            patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,img)
            medie_patch=np.mean(patch)
            gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
            T, img_threshOtsu = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
            medie_img = np.mean(img_threshOtsu)
            # show_img(img_threshOtsu,"thresh")
            # show_img(patch,"patch")
            # if 20 < medie_patch < medie_img:
            cv.imwrite(str(templates_path+str(index)+".jpg"),patch)
            index += 1

### Check the neighbors of the found piece in order to template match with only the possible ones

In [221]:
def check_possible_operations(i,j,dir_i,dir_j,game_matrix):
    possible_digits = []
    if i >= 0 and i < 14 and j >= 0 and j < 14 and  game_matrix[i,j] != -1:
        if i + dir_i >= 0 and i + dir_i < 14 and j +dir_j >= 0 and j + dir_j < 14 and game_matrix[i + dir_i,j + dir_j] != -1:
            possible_digits.append(game_matrix[i + dir_i,j + dir_j] + game_matrix[i,j])

            possible_digits.append(abs(game_matrix[i + dir_i,j + dir_j] - game_matrix[i,j]))

            possible_digits.append(game_matrix[i + dir_i,j + dir_j] * game_matrix[i,j])

            if game_matrix[i + dir_i,j + dir_j] % game_matrix[i,j] == 0:
                possible_digits.append(game_matrix[i + dir_i,j + dir_j] // game_matrix[i,j])

            elif  game_matrix[i,j] % game_matrix[i + dir_i,j + dir_j] == 0:
                possible_digits.append(game_matrix[i,j] // game_matrix[i + dir_i,j + dir_j])
    
    return possible_digits

### Check if the possible piece values are in the available pieces

In [223]:
def get_possible_digits(i,j,game_matrix,available_digits):
    possible_digits = []

    possible_digits.append(check_possible_operations(i-1,j,-1,0,game_matrix))
    possible_digits.append(check_possible_operations(i,j+1,0,1,game_matrix))
    possible_digits.append(check_possible_operations(i+1,j,1,0,game_matrix))
    possible_digits.append(check_possible_operations(i,j-1,0,-1,game_matrix))

    possible_digits = sum(possible_digits,[]) #flatten the array

    for digit in possible_digits:
        if digit in available_digits.keys():
            if available_digits[digit] == 0:
                possible_digits.remove(digit)
        else:
            possible_digits.remove(digit)


    return possible_digits,available_digits

### Perform template matching between the images from the templates folder and the new found piece

In [224]:
def template_match(patch,possible_digits):
    templates = os.listdir(templates_path)
    
    best_min_val = 99999999
    best_max_val = 0
    digit = -1

    top, bottom, left, right = 50, 50, 50, 50

    patch = cv.copyMakeBorder(patch,top, bottom, left, right,borderType=cv.BORDER_CONSTANT,value=[0, 0, 0])

    # show_img(patch,"patch")

    for template_name in templates:
        # print(template[:-4])
        if int(template_name[:-4]) in possible_digits:

            template = cv.imread(templates_path + template_name,cv.IMREAD_GRAYSCALE)

            _, template = cv.threshold(template, 50, 255, cv.THRESH_BINARY)

            possible_result = cv.matchTemplate(patch,template,cv.TM_SQDIFF)
            
            # possible_result = cv.matchTemplate(patch,template,cv.TM_CCOEFF) 

            min_val, max_val, _, _ = cv.minMaxLoc(possible_result)

            # print(min_val,max_val,template_name[:-4])

            if min_val < best_min_val:
                best_min_val = min_val
                digit = int(template_name[:-4])
            # if max_val > best_max_val:
            #     best_max_val = max_val
            #     digit = int(template_name[:-4])
                
    return digit       

In [ ]:
# img1_r=cv.resize(img1,(0,0),fx=0.20,fy=0.20)
# img2_r=cv.resize(img2,(0,0),fx=0.20,fy=0.20)
img1_r=img1.copy()
img2_r=img2.copy()
crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(img1_r,img2_r)

lines_horizontal,lines_vertical = generate_lines()

# write_digits_template(careu_img1,lines_horizontal,lines_vertical)

# for line in  lines_vertical : 
#     cv.line(careu_img2, line[0], line[1], (0, 0, 255), 5)
# for line in  lines_horizontal : 
#     cv.line(careu_img2, line[0], line[1], (0, 255, 0), 5)

# # cv.imshow("preproc_careu1",preproc_careu1)
# show_img(careu_img2,"careu_img2")
# # show_img(careu_img1,"preproc_careu2")
kernel = cv.getStructuringElement(cv.MORPH_RECT, (13,13))
result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

i,j = find_new_piece_position(result,lines_horizontal,lines_vertical)
# print(i+1,end="")
# print(chr(j+65))
# show_img(result,"result")
show_img(careu_img2[lines_horizontal[i][0][1]:lines_horizontal[i+1][0][1],lines_vertical[j][0][0]:lines_vertical[j+1][0][0],:],"patch")
preproc_patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img2)
show_img(preproc_patch,"preprocess_patch")
game_matrix = BASE_MATRIX.copy()
available_digits = DIGIT_DICT.copy()
possible_digits, available_digits = get_possible_digits(i,j,game_matrix,available_digits)
digit = template_match(preproc_patch,possible_digits)
available_digits[digit] -= 1
game_matrix[i,j] = digit
available_digits
# # show_img(careu_img2,"careu_img2")
# # show_img(crop_img1,"crop_img1")
# cv.imwrite("careu.png",careu_img2)
# # cv.imshow("careu_img1",preproc_careu2)
# # cv.imshow("result",result)
# # lines_vertical
# lines_horizontal

In [226]:
first_position = cv.imread("../imagini_auxiliare/01.jpg")
previous_img = first_position
training_path = '../antrenare/'
files=os.listdir(training_path)
lines_horizontal, lines_vertical = generate_lines()

game_matrix = BASE_MATRIX.copy()
available_digits = DIGIT_DICT.copy()

game = 1
ok = 0
for file in files:
    if file[-3:]=='jpg':
        current_img = cv.imread(training_path + file)

        if str(game) != file[0]:
            game += 1
            previous_img = first_position
            game_matrix = BASE_MATRIX.copy()
            available_digits = DIGIT_DICT.copy()

        crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(previous_img,current_img)

        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13,13))
        result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

        i,j = find_new_piece_position(result,lines_horizontal,lines_vertical)

        preproc_patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img2)

        possible_digits, available_digits = get_possible_digits(i,j,game_matrix,available_digits)

        digit = template_match(preproc_patch,possible_digits)

        available_digits[digit] -= 1
        
        game_matrix[i,j] = digit
        
        print(i+1,end="")
        print(chr(j+65),end=' ')
        print(digit)
        print(file)
        # show_img(preproc_patch,"preprocess_patch")
        
        
    elif file[-3:] == 'txt' and file.find("scores.txt") ==  -1  and file.find("turns.txt") == -1:
        contents = Path(training_path + file).read_text()

        if contents != str(i+1) + str(chr(j+65)) + " " + str(digit):
            print(contents)
            print(str(i+1) + str(chr(j+65)) + " " + str(digit))
            ok += 1
            # show_img(preproc_patch,"asad")


            # break

    
    previous_img = current_img
print(ok)
print(ok/200)

9H 8
1_01.jpg
10H 32
1_02.jpg
11H 40
1_03.jpg
12H 72
1_04.jpg
6H 8
1_05.jpg
5H 10
1_06.jpg
4H 80
1_07.jpg
3H 8
1_08.jpg
2H 10
1_09.jpg
1H 18
1_10.jpg
6G 3
1_11.jpg
6F 11
1_12.jpg
6E 14
1_13.jpg
6I 5
1_14.jpg
6J 3
1_15.jpg
6K 15
1_16.jpg
6L 12
1_17.jpg
7I 3
1_18.jpg
8I 7
1_19.jpg
7J 6
1_20.jpg
5J 9
1_21.jpg
7K 9
1_22.jpg
7L 3
1_23.jpg
6D 3
1_24.jpg
9I 10
1_25.jpg
10I 17
1_26.jpg
10J 49
1_27.jpg
8J 9
1_28.jpg
8K 63
1_29.jpg
9K 54
1_30.jpg
5L 36
1_31.jpg
4L 48
1_32.jpg
6M 27
1_33.jpg
4J 6
1_34.jpg
8F 7
1_35.jpg
3J 3
1_36.jpg
2J 2
1_37.jpg
1J 5
1_38.jpg
11I 7
1_39.jpg
12I 10
1_40.jpg
13I 70
1_41.jpg
14I 60
1_42.jpg
9G 2
1_43.jpg
10G 5
1_44.jpg
9F 10
1_45.jpg
10F 3
1_46.jpg
10E 8
1_47.jpg
10D 5
1_48.jpg
9E 8
1_49.jpg
11E 0
1_50.jpg
7I 3
2_01.jpg
7J 6
2_02.jpg
7K 9
2_03.jpg
7L 15
2_04.jpg
7M 6
2_05.jpg
7N 90
2_06.jpg
7F 3
2_07.jpg
8I 7
2_08.jpg
9I 21
2_09.jpg
9H 6
2_10.jpg
9G 27
2_11.jpg
10G 30
2_12.jpg
10H 24
2_13.jpg
10I 54
2_14.jpg
10F 6
2_15.jpg
6H 6
2_16.jpg
6G 4
2_17.jpg
6F 2
2_18.jpg


C:\Users\georg\AppData\Local\Temp\ipykernel_7316\739246660.py:11: RuntimeWarning: divide by zero encountered in scalar remainder
  if game_matrix[i + dir_i,j + dir_j] % game_matrix[i,j] == 0:
C:\Users\georg\AppData\Local\Temp\ipykernel_7316\739246660.py:12: RuntimeWarning: divide by zero encountered in scalar floor_divide
  possible_digits.append(game_matrix[i + dir_i,j + dir_j] // game_matrix[i,j])


5I 1
3_12.jpg
4I 1
3_13.jpg
4K 81
3_14.jpg
3J 10
3_15.jpg
7F 3
3_16.jpg
7E 4
3_17.jpg
7D 12
3_18.jpg
6H 6
3_19.jpg
5H 8
3_20.jpg
4H 48
3_21.jpg
4G 49
3_22.jpg
4F 1
3_23.jpg
8F 7
3_24.jpg
9F 10
3_25.jpg
10F 17
3_26.jpg
9H 2
3_27.jpg
10H 8
3_28.jpg
11H 10
3_29.jpg
12H 2
3_30.jpg
9I 2
3_31.jpg
10I 3
3_32.jpg
10G 11
3_33.jpg
10E 28
3_34.jpg
11F 7
3_35.jpg
10D 45
3_36.jpg
12F 10
3_37.jpg
6F 10
3_38.jpg
6G 4
3_39.jpg
8E 21
3_40.jpg
6E 25
3_41.jpg
6D 35
3_42.jpg
6C 10
3_43.jpg
7C 16
3_44.jpg
13F 3
3_45.jpg
7B 4
3_46.jpg
7A 20
3_47.jpg
5C 6
3_48.jpg
11I 5
3_49.jpg
11J 50
3_50.jpg
7I 3
4_01.jpg
7J 6
4_02.jpg
6H 6
4_03.jpg
5H 8
4_04.jpg
9G 4
4_05.jpg
8I 1
4_06.jpg
8J 5
4_07.jpg
7K 9
4_08.jpg
8K 5
4_09.jpg
4H 48
4_10.jpg
3H 40
4_11.jpg
2H 8
4_12.jpg
1H 32
4_13.jpg
10G 7
4_14.jpg
11G 28
4_15.jpg
12G 21
4_16.jpg
13G 7
4_17.jpg
14G 3
4_18.jpg
9H 6
4_19.jpg
10H 24
4_20.jpg
11H 30
4_21.jpg
12H 54
4_22.jpg
9K 45
4_23.jpg
8L 25
4_24.jpg
8M 5
4_25.jpg
8N 5
4_26.jpg
7L 3
4_27.jpg
7M 12
4_28.jpg
7N 15
4_29